<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [56]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error
import datetime
import re

In [58]:
!cd data/vw & unzip train.json.zip

Archive:  train.json.zip
  inflating: train.json              


In [3]:
!cd data/vw & unzip test.json.zip

Archive:  test.json.zip
  inflating: test.json               


Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [4]:
!head -1 data/vw/train.json > data/vw/train1.json

'head' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
with open('data/vw/train1.json') as inp_json:
    first_json = json.load(inp_json)

In [4]:
first_json.keys()

dict_keys(['_id', '_timestamp', 'url', 'domain', 'published', 'title', 'content', 'polling', 'post_id', 'flags', 'hubs', 'flow', 'tags', 'author', 'link_tags', 'meta_tags'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [5]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [6]:
first_json['_timestamp']

1493192186.0903192

In [7]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [8]:
first_json['domain']

'habrahabr.ru'

In [9]:
first_json['published']

{'$date': '2006-07-15T01:48:00.000Z'}

In [10]:
first_json['title']

'Самопроизвольное разлогинивание'

In [11]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [12]:
first_json['polling']

In [13]:
first_json['post_id']

7

In [14]:
first_json['flags']

[]

In [15]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [16]:
first_json['flow']

In [17]:
first_json['tags']

['логин', 'login']

In [18]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [19]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [20]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [5]:
train_target = pd.read_csv('data/vw/train_target.csv',
                          index_col='url')

In [6]:
train_target.head()

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000


Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину: постройте признак content_len – длина текста в миллионах символов.
Также постройте признаки: час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [23]:
!head -2 ../../data/habr_train.vw

0.6931470000000001 |title Самопроизвольное разлогинивание |tags логин login |domain habrahabr.ru |flow None |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1
1.0986120000000001 |title Stand-along cообщества против сообществ в рамках социальных сетей |tags сообщества интернет-сообщество социальные сети нишевой бренд |domain geektimes.ru |flow None |author @AlexBruce |hubs Чёрная дыра |num content_len:0.0 month:7 hour:14


In [11]:
train_target['target']['https://habrahabr.ru/post/7/']

0.69314700000000007

In [ ]:
Counter([])

In [103]:
counter = Counter(data_json['content'].split())
counter.get

<function Counter.get>

In [120]:
from collections import Counter
def json2vw(data_json, useTarget = True):
    counter = Counter(re.findall('\w{5,}',data_json['content']))
    content = sorted(counter, key=counter.get, reverse=True)[:10]
    s = u'{target} |title {title} |content {content} |tags {tags} |domain {domain} |flow {flow} |author {author} |hubs {hubs} |num content_len:{content_len} month:{month} hour:{hour}\n'\
    .format(target = train_target['target'][data_json['url']], 
            title = ' '.join(re.findall('\w{3,}', data_json['title'])),
            content = ' '.join(content),
            tags = ' '.join(re.findall('\w{3,}',' '.join(data_json['tags']))), #да, именно так)))
            domain = data_json['domain'], 
            flow = data_json['flow'], 
            author = data_json['author']['nickname'], 
            hubs = ' '.join([re.search('\w+',h['title']).group(0) for h in data_json['hubs']]), 
            content_len = len(data_json['content'])/10**6,
            month = datetime.datetime.strptime(data_json['published']['$date'], "%Y-%m-%dT%H:%M:%S.%fZ").month,
            hour = datetime.datetime.strptime(data_json['published']['$date'], "%Y-%m-%dT%H:%M:%S.%fZ").hour)
    return s

In [123]:
i = 0
with open('data/vw/train.json', encoding='utf-8') as inp_json, \
     open('data/vw/habr_train.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        s = json2vw(data_json, True)
        #content = 
        #s = u'{target} |title {title} |tags {tags} |domain {domain} |flow {flow} |author {author} |hubs {hubs} |num content_len:{content_len} month:{month} hour:{hour}\n'\
        #.format(target = train_target['target'][data_json['url']], 
        #        title = ' '.join(re.findall('\w{3,}', data_json['title'])),
        #        tags = ' '.join(re.findall('\w{3,}',' '.join(data_json['tags']))), #да, именно так)))
        #        domain = data_json['domain'], 
        #        flow = data_json['flow'], 
        #        author = data_json['author']['nickname'], 
        #        hubs = ' '.join([re.search('\w+',h['title']).group(0) for h in data_json['hubs']]), 
        #        content_len = len(data_json['content'])/10**6,
        #        month = datetime.datetime.strptime(data_json['published']['$date'], "%Y-%m-%dT%H:%M:%S.%fZ").month,
        #        hour = datetime.datetime.strptime(data_json['published']['$date'], "%Y-%m-%dT%H:%M:%S.%fZ").hour)
        out_vw.writelines(s)
        #i +=1
        #if i>1:
        #print(s)
        #break
        # Ваш код здесь

Widget Javascript not detected.  It may not be installed or enabled properly.


Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

In [87]:
with open('data/vw/test.json', encoding='utf-8') as inp_json, \
     open('data/vw/habr_test.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        s = json2vw(data_json, False)
        #s = u'{target} {utag}|title {title} |tags {tags} |domain {domain} |flow {flow} |author {author} |hubs {hubs} |num content_len:{content_len} month:{month} hour:{hour}\n'\
        #.format(target = 1, 
        #        utag = data_json['url'],
        #        title = ' '.join(re.findall('\w{3,}', data_json['title'])), 
        #        tags = ' '.join(re.findall('\w{3,}',' '.join(data_json['tags']))), #да, именно так)))
        #        domain = data_json['domain'], 
        #        flow = data_json['flow'], 
        #        author = data_json['author']['nickname'], 
        #        hubs = ' '.join([re.search('\w+',h['title']).group(0) for h in data_json['hubs']]), 
        #        content_len = len(data_json['content'])/10**6,
        #        month = datetime.datetime.strptime(data_json['published']['$date'], "%Y-%m-%dT%H:%M:%S.%fZ").month,
        #        hour = datetime.datetime.strptime(data_json['published']['$date'], "%Y-%m-%dT%H:%M:%S.%fZ").hour)
        out_vw.writelines(s)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [24]:
!head -2 ../../data/habr_test.vw

1 |title День Пи! |tags Пи Pi |domain geektimes.ru |flow None |author @Timursan |hubs Чёрная дыра |num content_len:0.0 month:3 hour:3
1 |title Скрипт для разбиения образов музыкальных CD на треки и конвертации в формат FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs Чёрная дыра |num content_len:0.01 month:3 hour:0


Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

In [75]:
!vw -k -d data/vw/habr_train.vw \
--loss_function classic -f habr_model.vw --quiet

In [88]:
!vw -i habr_model.vw -t -d data/vw/habr_test.vw \
-p data/vw/habr_predictions.txt

only testing
predictions = data/vw/habr_predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/vw/habr_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000994 0.000994            1            1.0   1.0000   0.9685        9
0.396612 0.792229            2            2.0   1.0000   1.8901       18
0.678740 0.960869            4            4.0   1.0000   2.3313       16
1.426011 2.173282            8            8.0   1.0000   2.3377       16
2.222731 3.019451           16           16.0   1.0000   0.0000       14
1.817297 1.411864           32           32.0   1.0000   3.3921       16
1.799915 1.782533           64           64.0   1.0000   1.3088       17
1.973282 2.146648          128          128.0   1.0000   2.5365       18
1.784566 1.595850          256          256.0   1.0000   3.2271       19
2

In [89]:
pred = pd.read_csv('data/vw/habr_predictions.txt', sep=' ', header=None)
pred.columns = ['target','url']
pred.to_csv('data/vw/sub.csv',index=False)
pred.head()

,target,url
0,0.968465,https://geektimes.ru/post/87455/
1,1.890073,https://geektimes.ru/post/87452/
2,0.613485,https://geektimes.ru/post/87459/
3,2.331294,https://habrahabr.ru/post/87461/
4,0.000000,https://habrahabr.ru/post/5754/


In [25]:
sample_sub = pd.read_csv('../../data/sample_submission.csv', 
                         index_col='url')

In [26]:
sample_sub.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [ ]:
your_submission = sample_sub.copy()
your_submission['target'] = # Ваш код здесь
your_submission.to_csv('submission.csv')

Для получения баллов в #mlcourse_open команда (из 1 человека) должна называться в точном соответствии с тем, как оно записано в рейтинге.